# Importing libraries and datasets

In [3]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix, save_npz
from sklearn import neighbors
from sklearn.cluster import KMeans

In [4]:
# import datasets:
genome_scores =  pd.read_csv('data/genome-scores.csv')
movies = pd.read_csv('data/movies.csv',  usecols = ["movieId"])
ratings = pd.read_csv('data/ratings.csv', usecols = ["movieId", "userId", "rating"])

# Preprocessing

In [ ]:
# Count movies in the movies df
tmm = movies["movieId"].nunique()
# Count movies in the matings df
tmr = ratings["movieId"].nunique()
# Count movies in the genome_score df
tmg = genome_scores["movieId"].nunique()
print("The total number of movies in movies, ratings and genome score dataframes, respectively :", tmm, tmr, tmg)

In [5]:
# Create a movie - user - rating data frame with movies present in all the dataframes
tagged_movies = pd.DataFrame(genome_scores['movieId'].value_counts()).index
mov_rat = pd.merge(movies, ratings, on="movieId")

In [6]:
final_df = mov_rat[mov_rat["movieId"].isin(tagged_movies)]

In [7]:
#Reducing the dataframe by removing unpopular movies and anactive 
#Shringking movies
gf = pd.DataFrame(final_df['movieId'].value_counts())
rare_movies = gf[gf['movieId'] <= 1000].index
final_df = final_df[~final_df["movieId"].isin(rare_movies)]
print('Out of total of ', gf.shape[0] , ' movies, ', rare_movies.shape[0], ' are considered rare and will be removed.')
print('The final number of movies is ', final_df["movieId"].nunique())

#Shringking users
udf = pd.DataFrame(final_df['userId'].value_counts())
lazy_users = udf[udf['userId'] <= 500].index
final_df = final_df[~final_df["userId"].isin(lazy_users)]
print('Out of total of ', udf.shape[0] , ' users, ', lazy_users.shape[0], ' are considered lazy and will be removed.')
print('The final number of users is ', final_df["userId"].nunique())

# Create the user->movie sparse rating matrix. Fill the NA with zeros
pivot = final_df.pivot_table(index="userId", columns="movieId", values="rating")
pivot.fillna(0,inplace=True)

#Estimate sparsity
sparsity = 1.0 - ( np.count_nonzero(pivot) / float(pivot.size) )
print("The resulting sparcity of the matrix is:", sparsity)

#Create non-sparce dataset
csr_data = csr_matrix(pivot.values)

Out of total of  10370  movies,  7214  are considered rare and will be removed.
The final number of movies is  3156
Out of total of  138493  users,  132651  are considered lazy and will be removed.
The final number of users is  5842
The resulting sparcity of the matrix is: 0.7548389812159577


In [9]:
final_df.to_csv('matrices/user_movie_rating.csv', index=False) 

In [8]:
pivot.to_csv('matrices/pivot.csv', index=True, header="userId")

In [10]:
save_npz("matrices/sparse_ratings.npz", csr_data)

# Modelling
Below we execute the following steps:
* Randomly choosing a target user fo whom recommendations will be generated
* Pre-clustering of users using KMeans model
* Creating a new matrix corresponding with users in the target user cluster
* Defining the Mean User Vector for the working data matrix
* Training the Nearest Neighbors model on the Mean User Vector
* Processing the results of the NN modeling: collecting, aggregating, filtering results and presenting recommendations

## Pre-clustering with KMeans

In [ ]:
# Pre-clustering users with K-means
# 20 clusters
kmeans = KMeans(n_clusters=20, algorithm='lloyd', n_init='auto').fit(csr_data) # Demands lots of resources
labels = kmeans.labels_ 
unique, counts = np.unique(labels, return_counts=True)

print("The number of users per class:\n")
for u, c in zip(unique, counts):
    print(u, c)

In [ ]:
#Load sparce crr user-movie utility matrix
print(csr_data)

In [ ]:
# Let's randomly choose an existing user
user_index = np.random.choice(csr_data.shape[0])    
print('The target user ', user_index, ' belongs to the cluster number', labels[user_index])

# Creating the csr matrix only for usrs of the cluster of the target user
# get indices of the users in the selected cluster
cluster_user_indices = []
for i,l in enumerate(labels):
    if l == labels[user_index]:
        cluster_user_indices.append(i)
        
# create a new csr_data only with the users from the cluster
mask = np.zeros(csr_data.shape[0], dtype=bool)
mask[cluster_user_indices] = True

#csr_data_cluster = csr_data[mask]
csr_data_cluster = csr_data 

# When shrinking the matrix, the index of the target user changes to the new_user_index
new_user_index = np.nonzero(np.array(cluster_user_indices) == user_index)[0][0]
print('The new index of the target user is ', new_user_index)

# Calculating the mean users' ratings
#def calculate_mean_user_vector(csr_data):
#    csr_mean = csr_data.sum(axis=1)/csr_data.getnnz(axis=1)
#    mean_user_vector = np.asarray(csr_mean)
#    flattened_reshaped = mean_user_vector.mean(axis=1).flatten().reshape(-1, 1)
#    return mean_user_vector.reshape(-1), flattened_reshaped

#mean_user_vector, model_mean_user_vector = calculate_mean_user_vector(csr_data_cluster)

## Nearest Neighbors model

In [ ]:
# Create NN object and fit it with the mean user vector

number_of_closest_users = 150
nn = neighbors.NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=number_of_closest_users)
nn.fit(csr_data_cluster) 
number_of_most_closest_users = 20 # for visualization purpose only
nn_sub = neighbors.NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=number_of_most_closest_users)
nn_sub.fit(csr_data_cluster)

# Find the nearest neighbors for the target user (e.g., User1)

mask = np.zeros(csr_data_cluster.shape[0], dtype=bool)
mask[new_user_index] = True
target_user_row = csr_data_cluster[mask]

distances, indices = nn.kneighbors(target_user_row)
distances_sub, indices_sub = nn_sub.kneighbors(target_user_row)

In [ ]:
# Exercise: visualize the results as graph
import networkx as nx

G = nx.Graph()
G.add_node(new_user_index)

closest_users = indices.reshape(-1)
closest_users_sub = indices.reshape(-1)[1:number_of_most_closest_users +1]
closest_users_remaining = indices.reshape(-1)[number_of_most_closest_users+1 :]

In [ ]:
print(len(closest_users), closest_users)
print(len(closest_users_sub), closest_users_sub)

In [ ]:
G.add_nodes_from(indices.reshape(-1)[1:])

weights = 1/(distances.reshape(-1)[1:])

G.add_weighted_edges_from(list((new_user_index, n, w) for n,w in zip(closest_users[1:],weights)))

pos = nx.fruchterman_reingold_layout(G)

plt.figure(figsize = (10, 10))
ax = plt.axes()
ax.set_facecolor("white")
plt.title("Visualization of the results of the NN algo")

nx.draw_networkx_nodes(G, pos, node_size = 100, nodelist = closest_users_remaining, node_color = "#ff5e33")
nx.draw_networkx_nodes(G, pos, node_size = 100, nodelist = closest_users_sub, node_color = "#33ff83", label= str(number_of_most_closest_users) + " closest users")
nx.draw_networkx_nodes(G, pos, node_size = 200, nodelist = [new_user_index], node_color = "#ffffff")
nxlabels = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edges(G, pos, alpha = 0.3, edge_color = "#48dbc8")
#nx.draw_networkx_edge_labels(G,pos,edge_labels=nxlabels)
#nx.draw_networkx_labels(G,pos,font_size=10, labels={user_index: user_index})
nx.draw_networkx_labels(G,pos,font_size=10)
plt.grid(visible=False)
plt.legend()
plt.show()

## Postprocessing

In [ ]:
# Collecting ratings from the closers users
def keep_rows_csr(mat, indices):
    """
   Keep the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    #if not isinstance(mat, scipy.sparse.csr_matrix):
     #   raise ValueError("works only for CSR format -- use .tocsr() first")
    indices = indices.flatten()
    mask = np.zeros(mat.shape[0], dtype=bool)
    mask[indices] = True
    return mat[mask]
    

# a csr matrix with the closest users only
csr_data_closests_users = keep_rows_csr(csr_data_cluster, indices)

# Aggregating the ratings from the closest users
# calculate the averaged rating of the movies given by the neiboghrs

def closest_users_average_ratings(mat):
    mat_array = mat.toarray()
    mat_array[mat_array == 0] = np.nan
    av_ratings = np.nanmean(mat_array, axis=0)
    df = pd.DataFrame(data ={'movieId': , 'rating': av_ratings })
    
    return df


rating_aggregation = closest_users_average_ratings(csr_data_closests_users)

# rank them by sorting
movies_indices_sorted_desc = np.argsort(rating_aggregation)[::-1]
best_20_movies = movies_indices_sorted_desc[:20]

# Filtering:
# get the user movies row
mask = np.zeros(csr_data_cluster.shape[0], dtype=bool)
mask[new_user_index] = True
target_user_row = csr_data_cluster[mask].toarray().flatten()

# get indices of the unwatched movies
unwatched_indices = np.nonzero(target_user_row == 0)

# get indices of the watched movies
watched_indices = np.nonzero(target_user_row != 0)

# keep only unwatches movie indices
filter_arr = []
for element in movies_indices_sorted_desc:
  if element in unwatched_indices[0]:
    filter_arr.append(True)
  else:
    filter_arr.append(False)

movies_to_watch_unfiltered = movies_indices_sorted_desc
movies_to_watch = movies_indices_sorted_desc[filter_arr]

# Provide personalised recommendations
watched_indices = np.nonzero(target_user_row != 0)

# get intersection between watched_indices and recommender indices
intersection_indices = np.intersect1d(watched_indices[0],best_20_movies[0])

print('User watched ', len(watched_indices[0]), ' movies')
print('System recommends ', len(best_20_movies), ' movies')
print(len(intersection_indices), ' movies are in common')

def get_movie_titles_by_indices(indices):
    titles = []
    for i in indices:
        titles.append(pivot.columns[i])
    return titles

print("User " + str(user_index) + " watched and rated the following movies:\n")

for index, movie in enumerate(get_movie_titles_by_indices(watched_indices)[:20]):
    print(str(index+1), ":", str(movie))
    
print("\n")
print("Recommendation for User " + str(user_index) + ":\n")
for index, movie in enumerate(get_movie_titles_by_indices(best_20_movies)):
    print(str(index+1), ":", str(movie))

In [ ]:
var = 5

In [ ]:
def text_function(x):
    return x + var

In [ ]:
print(text_function(6))